In [1]:
#essential libraries
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt   
import pickle as pkl
import shelve
import re
from collections import Counter, defaultdict, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics#v_measure_score
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy
import random

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB


In [2]:
def save_fig(fig, name, dpi=400):
	pp = PdfPages(name)
	pp.savefig(fig, bbox_inches='tight', pad_inches=0, dpi=dpi)
	pp.close()

In [3]:
def del_indices_list(srcList, indices):
    copyList = deepcopy(srcList)
    cnt = 0
    for index in indices:
        del copyList[index-cnt]
        cnt+=1
    return copyList

In [4]:
def find_index_sensor_list(sensorList, srcid):
    for index, sensor in enumerate(sensorList):
        if sensor['source_id']==srcid:
            return index

In [5]:
def find_cluster_from_srcid(clusterDict, srcid):
    for clusterNum, clusterList in clusterDict.iteritems():
        if srcid in clusterList:
            return clusterNum
    return None

In [6]:
is_substr = lambda s, strings: all(s in x for x in strings)
def long_substr(data):
    substr = ''
    if len(data) > 1 and len(data[0]) > 0:
        for i in range(len(data[0])):
            for j in range(len(data[0])-i+1):
                if j > len(substr) and is_substr(data[0][i:i+j], data):
                    substr = data[0][i:i+j]
    return substr

In [7]:
#Import shelve dictionary containing all bacnet devices
sensors_dict = shelve.open('metadata/bacnet_devices.shelve','r')

In [8]:
#device_list filters the NAE for a particular building. This is currently manual.
# EBU3B
buildingName = 'ap_m'
naeDict = dict()
naeDict['bonner'] = ["607", "608", "609", "557", "610"]
naeDict['ap_m'] = ['514', '513','604']
naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
naeDict['ebu3b'] = ["505", "506"]

device_list = naeDict[buildingName]

labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
#labeledFile = 'metadata/bonner_sensor_types_location.xlsx'
#labeledFile = 'metadata/ap_m_with_location.csv'
#labeledFile = 'metadata/bio_med_lib_sensor_types_location.xlsx'
resultFile = labeledFile.replace('.xlsx', '_mislocated.xlsx')
resultFile = labeledFile.replace('.csv', '_mislocated.xlsx')
resultFile = resultFile.replace('metadata', 'result')
dendoFile = labeledFile.replace('sensor_types_location.xlsx', 'dendo.pdf')
dendoFile = labeledFile.replace('sensor_types_location.csv', 'dendo.pdf')
dendoFile = dendoFile.replace('metadata', 'result')

samplePointDict = dict()
samplePointDict['bonner'] = [# RM-2001
    '607_2_3000761', '607_0_3000757', '607_1_3000763', '607_2_3000758', '607_2_3000759', '607_2_3000760', 
    '607_2_3000764', '607_2_3000765', '607_2_3000766', '607_2_3000767', '607_2_3000772', '607_2_3002217', 
    '607_5_3000768', '607_5_3000769', 
    # RM-2214
    '607_2_3000555', '607_0_3000544', '607_0_3000546', '607_0_3000559', '607_1_3000550', 
    '607_1_3000554', '607_1_3000560', '607_2_3000545', '607_2_3000551', '607_2_3000552', '607_2_3000553', 
    '607_2_3000556', '607_2_3000557', '607_2_3000558', '607_2_3000561', '607_2_3000562', 
    '607_3_3000547', '607_19_3000548', '607_19_3000549', '607_19_3000563'
                        ]

samplePointDict['ebu3b'] = [# RM-2150
    '506_1_3000493', '506_0_3000485', '506_0_3000488', '506_0_3000489', '506_0_3000490', '506_0_3000491', 
    '506_1_3000492', '506_1_3000494', '506_1_3000495', '506_1_3000496', '506_1_3000497', '506_1_3000498', 
    '506_1_3000499', '506_1_3000500', '506_1_3015651', '506_3_3000501', '506_14_3000504', '506_1_3001889', 
    # RM-4205
    '506_0_3001881',  '506_0_3001884', '506_0_3001885', '506_0_3001886', '506_0_3001887', '506_1_3001888', 
    '506_1_3001890', '506_1_3001891', '506_1_3001892', '506_1_3001893', '506_1_3001894', '506_1_3001895', 
    '506_1_3001896', '506_1_3015750', '506_14_3001899'
]
samplePointDict['ap_m'] = [ #RM-6161'
    '604_0_3012403', '604_0_3012404', '604_0_3012413', '604_0_3012415', '604_0_3012416', '604_1_3012405',
    '604_1_3012406', '604_1_3012407', '604_1_3012408', '604_1_3012409', '604_1_3012410', '604_1_3012411',
    '604_1_3012412', '604_1_3012414', '604_4_3012402',
    # RM-3108
    '513_0_3007058', '513_0_3007060', '513_0_3007065', '513_0_3007072', '513_0_3007076', '513_0_3007077', 
    '513_1_3007063', '513_1_3007064', '513_1_3007066', '513_1_3007069', '513_1_3007070', '513_1_3007074', 
    '513_14_3007062', '513_14_3007068'
    ]


                           
                           
samplePointList = samplePointDict[buildingName]

#tsLearningLabelDict = dict()
## 1=='Temperature Setpoint'
#tsLearningLabelDict['ebu3b'] = [  
#                    1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, # RM-2150
#                    1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0    # RM-4205   
#]
##tsLearningLabelDict['bonner'] = [
#    1,0,0,0,0,0,0,0,0,0,
#    0,0,0,0,1,0,0,0,0,0,
#    0,0,0,0,0,0,0,0,0,0,
#    0,0,0,0    # RM-4205   
#]
#tsLearningLabelDict['ap_m'] = [0]
#tsLearningLabelList = tsLearningLabelDict[buildingName]
#assert(len(tsLearningLabelList)==len(samplePointList))


vavSampleDict = dict()
nonvavSampleDict = dict()
vavSampleDict['ebu3b'] = ['506_0_3000488', '506_0_3002877', '506_0_3000886']
nonvavSampleDict['ebu3b'] = ['505_1_3000860', '505_0_3014673']
vavSampleDict['bonner'] = ['557_1_3002743', '608_0_3000489', '607_2_3000257'] #109 17 180
nonvavSampleDict['bonner'] = ['557_1_3003966', '557_0_3004593']
vavSampleDict['ap_m'] = ['604_0_3013537', '513_1_3004863', '513_0_3007065']
nonvavSampleDict['ap_m'] = ['513_1_3011367', '604_0_3013295']
fixedVavSampleList = vavSampleDict[buildingName]
fixedNonVavSampleList = nonvavSampleDict[buildingName]

In [9]:
print labeledFile

metadata/ap_m_sensor_types_location.csv


In [10]:
#Get Labeled metadata

labeledList = list()

#for filename in filenameList:
with open(labeledFile, 'rb') as fp:
    #truthDF = pd.read_excel(fp)
    truthDF = pd.DataFrame.from_csv(fp)
    
    #reader = csv.reader(fp, delimiter=',')
    #indices = reader.next()
    #Check if local indices are same as mainIndices
    #for index in indices:
        #if not index in mainIndices:
        #    print indices
        #    print index
        #    print ("indices are wrong. Please do fix the column names in the file: %s", labeledFile)
        #    sys.exit("Error message")
for row in truthDF.iterrows():
    newMetadataDict = dict()
    newMetadataDict['source_identifier'] = row[1]['Unique Identifier']
    newMetadataDict['location'] = row[1]['Location']
    newMetadataDict['point_type'] = row[1]['Ground Truth Point Type']
    newMetadataDict['equip_type'] = row[1]['Equipment Type']
    labeledList.append(newMetadataDict)
labeledMetadata = pd.DataFrame(labeledList)

In [11]:
#Parse the data in the dictionary as filtered by device_list
#Gives us a sensor_list with sensor information of a building

def extract_numbers(sentence):
    result = re.findall('(\d+\s?-?\s?\d+)|(\d+)', sentence)
    result = list(sum(result,()))
    while '' in result:
        result.remove('')
    for i, word in enumerate(result):
        if ' ' in word:
            result[i] = result[i].replace(' ', '-')
    return result


sensor_list = []
names_num_list = []
names_str_list = []
names_num_listWithDigits = [] 
sensor_type_namez=[]
desc_list = []
unit_list = []
type_str_list = []
type_list = []
jci_names_num_list = []
jci_names_str_list = []
source_id_set = set([])
for nae in device_list:
    device = sensors_dict[nae]
    h_dev = device['props']
    for sensor in device['objs']:
        h_obj = sensor['props']
        source_id = str(h_dev['device_id']) + '_' + str(h_obj['type']) + '_' + str(h_obj['instance'])
        
        if h_obj['type'] not in (0,1,2,3,4,5,13,14,19):
            continue
        
        if source_id in source_id_set:
            continue
        else:
            source_id_set.add(source_id)
        
        #create individual lists
        #remove numbers from names because they do not indicate type of sensor
        names_num_listWithDigits.append(sensor['jci_name']) 
        sensor_type_namez.append(sensor['sensor_type'])
        names_str_list.append(''.join([c for c in sensor['name'] if not c.isdigit()]))
        #names_num_list.append(' '.join(re.findall('\d+', sensor['name'])))
        names_num_list.append(' '.join(extract_numbers(sensor['name'])))
        
        desc_list.append(''.join([c for c in sensor['desc'] if not c.isdigit()]))
        #desc_list.append(' '.join(re.findall('\d+', sensor['desc'])))
        #jci_names_str_list.append(''.join([c for c in sensor['jci_name'] if not c.isdigit()]))
        jci_names_str_list.append(' '.join(re.findall('[a-zA-Z]+', sensor['jci_name'])))
        #jci_names_num_list.append(' '.join(re.findall('\d+', sensor['jci_name'])))
        jci_names_num_list.append(' '.join(extract_numbers(sensor['jci_name'])))
        
        #convert string to dictionary for categorical vectorization
        unit_list.append({str(sensor['unit']):1})
        type_str_list.append({str(h_obj['type_str']):1})
        type_list.append({str(h_obj['type']):1})
        
        #create a flat list of dictionary to avoid using json file
        sensor_list.append({'source_id': source_id, 
                            'name': sensor['name'], 
                            'description': sensor['desc'],
                            'unit': sensor['unit'],
                            'type_string': h_obj['type_str'],
                            'type': h_obj['type'],
                            #'device_id': h_obj['device_id'],
                            'jci_name': sensor['jci_name'],
                            #add data related characteristics here
                        })
sensor_df = pd.DataFrame(sensor_list)
#sensor_df = sensor_df.set_index('source_id')
#sensor_df = sensor_df.groupby(sensor_df.index).first()
origSensorList = deepcopy(sensor_list)

In [12]:
#print names_num_list[find_index_sensor_list(sensor_list, '557_1_3003777')]
#print jci_names_str_list[find_index_sensor_list(sensor_list, '557_1_3003777')]
#print names_num_list[find_index_sensor_list(sensor_list, '557_1_3003776')]
#print jci_names_str_list[find_index_sensor_list(sensor_list, '557_1_3003776')]

In [13]:
#Create a bag of words from sensor string metadata. Vectorize so that it can be used in ML algorithms.
#namevect = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
nameNumVect = CountVectorizer(token_pattern='\d+')
nameNumBow = scipy.sparse.coo_matrix(nameNumVect.fit_transform(names_num_list))
nameStrVect = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
nameStrBow = scipy.sparse.coo_matrix(nameStrVect.fit_transform(names_str_list))

descVect = CountVectorizer() 
descBow = scipy.sparse.coo_matrix(descVect.fit_transform(desc_list))

unitVect = DictVectorizer() 
unitBow = scipy.sparse.coo_matrix(unitVect.fit_transform(unit_list))

type_str_vect = DictVectorizer() 
typeStrBow = scipy.sparse.coo_matrix(type_str_vect.fit_transform(type_str_list))

typevect = DictVectorizer() 
typeBow = scipy.sparse.coo_matrix(typevect.fit_transform(type_list))

jciNumVect = CountVectorizer(token_pattern='\d+') 
jciNumBow = scipy.sparse.coo_matrix(jciNumVect.fit_transform(jci_names_num_list))
jciStrVect = CountVectorizer() 
jciStrBow = scipy.sparse.coo_matrix(jciStrVect.fit_transform(jci_names_str_list))

feature_set = jciStrVect.get_feature_names()+ \
              descVect.get_feature_names()+ \
              unitVect.get_feature_names()+ \
              type_str_vect.get_feature_names()+ \
              typevect.get_feature_names()

origJciNumBow = deepcopy(jciNumBow)
origNameNumBow = deepcopy(nameNumBow)
origDescbow = deepcopy(descBow)
origJciStrBow = deepcopy(jciStrBow)
origNameStrBow = deepcopy(nameStrBow)
origUnitBow = deepcopy(unitBow)
origTypeBow = deepcopy(typeBow)
origTypeStrBow = deepcopy(typeStrBow)

In [14]:
# filter_bow removes words occuring too many times from dictionary
def filter_bow(bow):
    wordsCount = bow.sum(axis=0)
    
    manyWordsIndices = [i for i, x in enumerate((wordsCount>40).tolist()[0]) if x == True]
    #print manyWordsIndices
    properWordsIndices = [i for i in range(0,wordsCount.size) if not (i in manyWordsIndices)]
    #print properWordsIndices
    return scipy.sparse.csr_matrix(bow.toarray()[:,properWordsIndices])

def tfidf_bow(bow):
    transformer = TfidfTransformer()
    return transformer.fit_transform(bow)#.toarray()
    

In [15]:
#Filter noise words from bow

filterFunc = lambda x: tfidf_bow(x)
#filterFunc = lambda x: filter_bow(x)

#jcibow = filter_bow(origJciNumBow)
#namebow = filter_bow(origNameNumBow)
#descbow = filter_bow(origDescbow)


#jciNumBow = tfidf_bow(origJciNumBow)
#jciStrBow = tfidf_bow(origJciStrBow)
#nameNumBow = tfidf_bow(origNameNumBow)
#nameStrBow = tfidf_bow(origNameStrBow)
#unitBow = tfidf_bow(origUnitBow)
#typeBow = tfidf_bow(origTypeBow)
#typeStrBow = tfidf_bow(origTypeStrBow)

jciNumBow = filterFunc(origJciNumBow)
jciStrBow = filterFunc(origJciStrBow)
nameNumBow = filterFunc(origNameNumBow)
nameStrBow = filterFunc(origNameStrBow)
unitBow = filterFunc(origUnitBow)
typeBow = filterFunc(origTypeBow)
typeStrBow = filterFunc(origTypeStrBow)


In [16]:
print type(jciStrBow)
print jciStrBow

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 126)	0.591312045924
  (0, 100)	0.802623981882
  (0, 10)	0.078388826076
  (1, 132)	0.990167381579
  (1, 10)	0.139887656554
  (2, 187)	0.697547754977
  (2, 174)	0.703574190939
  (2, 10)	0.135685251118
  (3, 174)	0.981907310385
  (3, 10)	0.1893621763
  (4, 187)	0.835733027966
  (4, 70)	0.524521708133
  (4, 10)	0.162564706084
  (5, 187)	0.835733027966
  (5, 70)	0.524521708133
  (5, 10)	0.162564706084
  (6, 206)	0.69681760136
  (6, 32)	0.709554014875
  (6, 10)	0.104777528177
  (7, 206)	0.670949074284
  (7, 33)	0.734608054824
  (7, 10)	0.100887786702
  (8, 206)	0.65807283621
  (8, 97)	0.746423951239
  (8, 10)	0.0989516410091
  :	:
  (4371, 18)	0.694833963706
  (4372, 170)	0.242626458666
  (4372, 134)	0.737617326504
  (4372, 39)	0.6301214813
  (4373, 170)	0.185503438675
  (4373, 124)	0.694833963706
  (4373, 18)	0.694833963706
  (4374, 170)	0.242626458666
  (4374, 134)	0.737617326504
  (4374, 39)	0.6301214813
  (4375, 170)	0.185503438675
  (4375, 124

In [17]:
finalNumBow = scipy.sparse.hstack([
                                 nameNumBow,
                                 #descbow,
                                 #unitbow,
                                 #type_str_bow,
                                 #typebow,
                                 jciNumBow
                                ]) 
bowNumArray = finalNumBow.toarray() # this is the bow for each sensor. 
origNumBowArray = deepcopy(bowNumArray)

finalStrBow = scipy.sparse.hstack([
                                 nameStrBow,
                                 descBow,
                                 unitBow,
                                 typeStrBow,
                                 typeBow,
                                 jciStrBow
                                ]) 
bowStrArray = finalStrBow.toarray() # this is the bow for each sensor. 
origStrBowArray = deepcopy(bowStrArray)



In [18]:
# List up sensors with nan location
nanIndices = list()
nanList = list()
for row in labeledMetadata.iterrows():
    location = row[1]['location']
    if type(location)==float:
        if math.isnan(location):
            nanList.append(row[1]['source_identifier'])
            continue
    pointType = row[1]['point_type']
    if pointType.lower()=='unknown':
        nanList.append(row[1]['source_identifier'])

# Remove sensors with zero vectors
zerovectorIndices = np.where(np.sum(origNumBowArray, axis=1) == 0)[0]
zerovectorList = [origSensorList[index]['source_id'] for index in zerovectorIndices]
bowNumArray = np.delete(origNumBowArray, zerovectorIndices, axis=0)
bowStrArray = np.delete(origStrBowArray, zerovectorIndices, axis=0)
sensor_list = del_indices_list(origSensorList, zerovectorIndices)

for index, sensor in enumerate(sensor_list):
    if sensor['source_id'] in nanList:
        nanIndices.append(index)
            
bowNumArray = np.delete(bowNumArray, nanIndices, axis=0)
bowStrArray = np.delete(bowStrArray, nanIndices, axis=0)
sensor_list = del_indices_list(sensor_list, nanIndices)
#print nanList
#print len(nanList)

filteredList = zerovectorList + [nan for nan in nanList if not nan in zerovectorList]
print len(bowNumArray)
print len(bowStrArray)
print len(sensor_list)

4170
4170
4170


In [19]:
print jciStrVect.vocabulary_
print jciStrBow.toarray()[1913]
print jciStrVect.inverse_transform(jciStrBow.toarray()[1913])
print jciStrBow.toarray()[1912]
print jciStrVect.inverse_transform(jciStrBow.toarray()[1912])

{u'alm': 6, u'feedback': 81, u'volts': 223, u'occhtgfl': 137, u'rcdb': 163, u'onochbia': 147, u'ahtg': 3, u'dt': 60, u'current': 48, u'bchw': 14, u'zone': 230, u'easf': 64, u'occ': 134, u'clg': 35, u'byp': 25, u'hall': 91, u'lvfd': 115, u'econdif': 69, u'curr': 47, u'east': 66, u'adj': 1, u'easp': 65, u'lftr': 105, u'supflosp': 200, u'emergency': 72, u'di': 51, u'db': 49, u'band': 13, u'fan': 79, u'stpt': 195, u'dx': 61, u'press': 159, u'dmd': 55, u'rftr': 169, u'repb': 167, u'dp': 57, u'vlv': 222, u'loss': 112, u'unoccbia': 216, u'uflr': 212, u'lowosa': 114, u'dpr': 58, u'ohmin': 143, u'level': 104, u'cmd': 39, u'humidity': 99, u'rvfd': 173, u'common': 44, u'ocmin': 139, u'chwdpstpt': 30, u'zn': 229, u'differential': 53, u'pmp': 155, u'navg': 128, u'ucmnc': 211, u'setpoint': 183, u'ef': 70, u'ea': 62, u'lvlco': 117, u'flt': 86, u'ebd': 67, u'static': 192, u'flsf': 85, u'wc': 227, u'supflo': 199, u'chwdp': 29, u'cmaxflo': 38, u'tmp': 208, u'sathtg': 180, u'capacity': 27, u'esp': 77, u'

In [20]:
# Hierarchical agglomerative clustering 
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier

#num_of_sensors = len(bowNumArray)
#a = np.array(bowNumArray[:num_of_sensors])
a = np.array(bowNumArray)
z = linkage(a,metric='cityblock',method='complete')

In [21]:
plt.close('all')
indexOffset = 2500
dendoA = a[indexOffset:indexOffset + 500]
dendoZ = linkage(dendoA, metric='cityblock',method='complete')
fig = plt.figure(figsize=(45,15))
ax = fig.add_subplot(111)
#dendroResult = dendrogram(z, ax=ax, truncate_mode='lastp', p=3000, color_threshold='default', leaf_font_size=4)
def llf(idx):
    srcid = sensor_list[indexOffset + idx]['source_id']
    return str(labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]['location'].iloc[0])
dendroResult = dendrogram(dendoZ, ax=ax, leaf_font_size=5, leaf_label_func=llf, leaf_rotation=90)
ax.set_ylim((-0.1,5))
save_fig(fig, dendoFile, dpi=5000)

In [22]:
# Input:A dictionary of lists, element want to search, Output: key corresponding to the value
def find_key_from_list_dict(d, val):
    for key, l in d.iteritems():
        if val in l:
            return key
    return None

In [23]:
#Clustering result.

b = hier.fcluster(z, 2, criterion='distance')

# Make a dict of lists. Key: room number, elements in the list: 
#roomDict = defaultdict(list)
roomDict = dict()
correctLabels = list()
for sensor in sensor_list:#[0:200]:
    sourceId = sensor['source_id']
    labeledSensor = labeledMetadata.loc[labeledMetadata['source_identifier']==sourceId]
    #try:
    room = labeledSensor['location'].get_values()[0]
    correctLabels.append(room)
    #except:
     #   print sourceId
    if room in roomDict.keys():
        roomDict[room].append(sourceId)
    else:
        roomDict[room] = [sourceId]
#print roomDict

clusterDict = defaultdict(list)
incorrectLabels = list()
for clusterNum, sensor in zip(b, sensor_list):
    clusterDict[clusterNum].append(sensor['source_id'])
    incorrectLabels.append(clusterNum)
print "#correctLabels: ", len(correctLabels)
print "#incorrectLabels: ", len(incorrectLabels)
print "Are they same?: ", len(correctLabels)==len(incorrectLabels)

#correctLabels:  4170
#incorrectLabels:  4170
Are they same?:  True


In [24]:
# Measure precision
#totalCnt = 0
#correctCnt = 0
#incorrectResult = dict()
#for clusterNum, clusterList in clusterDict.iteritems():
#    vote = defaultdict(int)
#    totalCnt += len(clusterList)
#    for sourceId in clusterList:
#        foundRoom = find_key_from_list_dict(roomDict, sourceId)
#        vote[foundRoom] += 1
#    correctCnt += max(vote.values())
    
#    # list incorrectLabel
#    currLabel = vote.keys()[vote.values().index(max(vote.values()))]
#    for sourceId in clusterList:
#        foundRoom = find_key_from_list_dict(roomDict, sourceId)
#        if foundRoom.lower() != currLabel.lower():
#            incorrectResult[sourceId] = currLabel
    
# Measure precision
totalCnt = 0
correctCnt = 0
incorrectResult = dict()
labelDict = defaultdict(list)
for clusterNum, clusterList in clusterDict.iteritems():
    vote = defaultdict(int)
    totalCnt += len(clusterList)
    for sourceId in clusterList:
        foundRoom = find_key_from_list_dict(roomDict, sourceId)
        vote[foundRoom] += 1
    correctCnt += max(vote.values())
    
    # list incorrectLabel
    currLabel = vote.keys()[vote.values().index(max(vote.values()))]
    labelDict[currLabel].append(clusterNum)
    for sourceId in clusterList:
        foundRoom = find_key_from_list_dict(roomDict, sourceId)
        if foundRoom.lower() != currLabel.lower():
            incorrectResult[sourceId] = currLabel
    print clusterNum, len(clusterList)
Counter([len(clusterList) for clusterList in clusterDict.values()])
for currLabel, clusterList in labelDict.iteritems():
    if len(clusterList)>1:
        lenList = [len(clusterDict[cluster]) for cluster in clusterList]
        correctCluster = clusterList[lenList.index(max(lenList))]
        for cluster in clusterList:
            if correctCluster!=cluster:
                for srcid in clusterDict[cluster]:
                    if not srcid in incorrectResult:
                        incorrectResult[srcid] = currLabel
                        

1 14
2 15
3 15
4 14
5 15
6 16
7 16
8 15
9 15
10 15
11 2
12 18
13 3
14 14
15 15
16 15
17 15
18 18
19 2
20 15
21 8
22 15
23 15
24 60
25 45
26 15
27 15
28 15
29 15
30 15
31 15
32 15
33 15
34 15
35 15
36 15
37 15
38 15
39 15
40 15
41 15
42 15
43 15
44 3
45 14
46 15
47 15
48 3
49 15
50 15
51 15
52 14
53 15
54 15
55 15
56 14
57 4
58 7
59 25
60 20
61 15
62 54
63 16
64 30
65 16
66 15
67 15
68 15
69 15
70 15
71 15
72 15
73 15
74 15
75 15
76 15
77 15
78 15
79 30
80 43
81 14
82 28
83 28
84 28
85 56
86 1
87 42
88 14
89 29
90 56
91 42
92 14
93 28
94 28
95 18
96 14
97 14
98 15
99 15
100 16
101 15
102 16
103 6
104 11
105 15
106 14
107 28
108 28
109 42
110 84
111 7
112 29
113 15
114 28
115 15
116 14
117 2
118 30
119 15
120 16
121 34
122 42
123 15
124 22
125 15
126 15
127 34
128 15
129 15
130 14
131 14
132 14
133 14
134 14
135 14
136 15
137 15
138 14
139 16
140 29
141 15
142 15
143 15
144 5
145 15
146 15
147 14
148 15
149 15
150 14
151 1
152 14
153 15
154 11
155 7
156 9
157 61
158 15
159 14
160 15
161 

In [25]:
#  Make bows for clusters.

clusterBowArray = np.zeros((len(clusterDict),len(bowStrArray[0])))
for clusterNum, clusterList in clusterDict.iteritems():
    
    clusterBow = np.zeros(len(bowStrArray[0]))
    for srcid in clusterList:
        sensorIdx = find_index_sensor_list(sensor_list, srcid)     
        clusterBow += bowStrArray[sensorIdx]
        
    clusterBowArray[clusterNum-1] = clusterBow

In [46]:
# Binary classifier for VAV clusters
random.seed()
randomSampleFlag = False
if randomSampleFlag:
    vavSampleList = list()
    nonVavSampleList = list()
    while len(vavSampleList)+len(nonVavSampleList)<5:
        sensorIdx = int(random.random()*len(sensor_list))
        sensor = sensor_list[sensorIdx]
        srcid = sensor['source_id']
        equipType = labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]['equip_type'].iloc[0]
        if equipType in ['vav', 'VAV'] and len(vavSampleList)<3:
            vavSampleList.append(srcid)
        elif not (equipType in ['vav', 'VAV']) and len(nonVavSampleList)<2:
            nonVavSampleList.append(srcid)
else:
    vavSampleList = fixedVavSampleList
    nonVavSampleList = fixedNonVavSampleList
print vavSampleList
print nonVavSampleList
        

indexList = list()
vavLearningLabelList = list()
for srcid in vavSampleList:
    indexList.append(find_cluster_from_srcid(clusterDict, srcid))
    vavLearningLabelList.append(1) # 1 == VAV
for srcid in nonVavSampleList:
    indexList.append(find_cluster_from_srcid(clusterDict, srcid))
    vavLearningLabelList.append(0) # 0 !=VAV
learningSampleBow = clusterBowArray[indexList]
print "Hello"
print indexList
print learningSampleBow


#model = RandomForestClassifier(n_estimators=400, random_state=0)
#model = OneClassSVM(nu=0.5, kernel="rbf")#, gamma=0.1)
#model = DPGMM(n_components = 17, covariance_type='full')
#model = KNeighborsClassifier(n_neighbors=3)
#model = LogisticRegression() #So far, the best
model = GaussianNB()  # By far the best.
#model = BernoulliNB()
#model = AdaBoostClassifier(n_estimators=100)
#model = MultinomialNB()
model.fit(X=learningSampleBow, y=vavLearningLabelList)
predProbs = model.predict_proba(clusterBowArray)
predLabels = model.predict(clusterBowArray).tolist()
vavDict = dict([(clusterNum, 'VAV') if vavLabel==1 else (clusterNum, '') for clusterNum, vavLabel in zip(clusterDict.keys(), predLabels)])
vavScoreDict = dict([(clusterNum, predProb) for clusterNum, predProb in zip(clusterDict.keys(), predProbs)])

['604_0_3013537', '513_1_3004863', '513_0_3007065']
['513_1_3011367', '604_0_3013295']
Hello
[2, 5, 96, 21, 59]
[[ 0.          0.81096019  0.65611657 ...,  0.          0.91606636  0.        ]
 [ 0.          0.81096019  0.65611657 ...,  0.          0.91606636  0.        ]
 [ 0.          0.81096019  0.65611657 ...,  0.          0.91606636  0.        ]
 [ 0.          0.81096019  0.65611657 ...,  0.          0.91606636  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [47]:
# VAV classification result.

correctVavCnt = 0
correctNonVavCnt = 0
incorrectVavCnt = 0
incorrectNonVavCnt = 0
vavAccuratePoints = 0
for (clusterNum, clusterList), predLabel in zip(clusterDict.iteritems(), predLabels):
    vote = 0
    for srcid in clusterList:
        equipType = labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]['equip_type'].iloc[0]
        if equipType=='vav' or equipType=='VAV':
            vote += 1
        else:
            vote -= 1
        if (equipType in ['vav', 'VAV'] and predLabel==1) or (not equipType in ['vav', 'VAV'] and predLabel==-1):
            vavAccuratePoints += 1
    if vote>0 and predLabel==1:
        correctVavCnt += 1
    elif vote<=0 and predLabel==0:
        correctNonVavCnt += 1
    elif vote<=0 and predLabel == 1:
        print "Wrong, classified as VAV: ", clusterNum
        incorrectVavCnt += 1
    else:
        print "Wrong, classified as nonVAV: ", clusterNum
        incorrectNonVavCnt += 1
        
vavPrecision = float(correctVavCnt)/float(correctVavCnt+incorrectVavCnt)
vavRecall = float(correctVavCnt)/float(correctVavCnt+incorrectNonVavCnt)
vavAccuracyPerPoint = float(vavAccuratePoints) / float(len(sensor_list))

print "vav precision: ", vavPrecision
print "vav recall: ", vavRecall
print 'vav accuracy per point: ', vavAccuracyPerPoint


print "Correct VAV #: ", correctVavCnt
print "Correct Non-VAV #: ", correctNonVavCnt
print "Incorrect VAV #: ", incorrectVavCnt
print "Incorrect Non-VAV #:", incorrectNonVavCnt

Wrong, classified as nonVAV:  4
Wrong, classified as nonVAV:  11
Wrong, classified as nonVAV:  12
Wrong, classified as nonVAV:  18
Wrong, classified as nonVAV:  24
Wrong, classified as nonVAV:  25
Wrong, classified as nonVAV:  42
Wrong, classified as nonVAV:  61
Wrong, classified as nonVAV:  64
Wrong, classified as nonVAV:  79
Wrong, classified as nonVAV:  80
Wrong, classified as nonVAV:  85
Wrong, classified as nonVAV:  86
Wrong, classified as nonVAV:  89
Wrong, classified as nonVAV:  90
Wrong, classified as nonVAV:  95
Wrong, classified as nonVAV:  110
Wrong, classified as nonVAV:  115
Wrong, classified as nonVAV:  151
Wrong, classified as nonVAV:  166
Wrong, classified as nonVAV:  170
Wrong, classified as nonVAV:  176
Wrong, classified as nonVAV:  192
Wrong, classified as nonVAV:  200
Wrong, classified as nonVAV:  212
Wrong, classified as nonVAV:  213
Wrong, classified as nonVAV:  214
Wrong, classified as nonVAV:  219
Wrong, classified as nonVAV:  229
Wrong, classified as nonVAV:  2

In [28]:
# Binary Classifier for Temperature Setpoint

indexList = list()
#tsTruthLabels = list()
tsLearningLabelList = list()
for srcid in samplePointList:
    index = indexList.append(find_index_sensor_list(sensor_list, srcid))
    if labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]['point_type'].iloc[0] in ['Temperature Setpoint', 'temperature setpoint']:
        tsLearningLabelList.append(1)
    else:
        tsLearningLabelList.append(0)
        #tsTruthLabels.append(0)
learningSampleBow = bowStrArray[indexList]
learningSampleBow = scipy.sparse.coo_matrix(learningSampleBow)
print tsLearningLabelList

typeTruthLabels = list()
for sensor in sensor_list:
    srcid = sensor['source_id']
    pointType = labeledMetadata[labeledMetadata['source_identifier']==srcid]['point_type'].iloc[0]
    typeTruthLabels.append(pointType)

#model = AdaBoostClassifier(n_estimators=100)
#model = RandomForestClassifier(n_estimators=400, random_state=0)
#model = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
#model = DPGMM(n_components = 17, covariance_type='full')
#model = KNeighborsClassifier(n_neighbors=3)
#model = LogisticRegression()
model = AdaBoostClassifier(n_estimators=100)
#model = MultinomialNB()
model.fit(X=learningSampleBow.toarray(), y=tsLearningLabelList)
predProbs = model.predict_proba(bowStrArray)
predLabels = model.predict(bowStrArray).tolist()
thresholdProb = 0.5
#print len(predProbs), predProbs.size
for i, (prob, label) in enumerate(zip(predProbs, predLabels)):
    if max(prob) < thresholdProb:
        predLabels[i] = None
tsPredictDict = dict([(sensor_list[i]['source_id'], predLabel) for i, predLabel in enumerate(predLabels)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [29]:
# Print result of the binary classifier for TS
print np.sum(np.asarray(predLabels)==1)
print(len(predLabels))
print(len(typeTruthLabels))
print np.sum(np.equal(predLabels, None))

correctTsNum = np.sum([((trueLabel.lower() in ['common setpoint', 'temperature setpoint']) and (predLabel==1)) for trueLabel, predLabel in zip(typeTruthLabels, predLabels)])
incorrectTsNum = np.sum([((trueLabel.lower() in ['common setpoint', 'temperature setpoint']) and (predLabel==0)) for trueLabel, predLabel in zip(typeTruthLabels, predLabels)])
correctNonTsNum = np.sum([(not (trueLabel.lower() in ['common setpoint', 'temperature setpoint']) and (predLabel==0)) for trueLabel, predLabel in zip(typeTruthLabels, predLabels)])
incorrectNonTsNum = np.sum([(not (trueLabel.lower() in ['common setpoint', 'temperature setpoint']) and (predLabel==1)) for trueLabel, predLabel in zip(typeTruthLabels, predLabels)])

print incorrectTsNum
print correctTsNum
tsPrecision = float(correctTsNum) / float(incorrectNonTsNum + correctTsNum)
tsRecall = float(correctTsNum) / float(incorrectTsNum + correctTsNum)
print 'TS precision: ', tsPrecision
print 'TS recall: ', tsRecall


103
4170
4170
0
1
103
TS precision:  1.0
TS recall:  0.990384615385


In [30]:
# Test Type Identification
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(labeledMetadata['point_type'].values)

typeTestLabels = list()
for sensor in sensor_list:
    #print sum(labeledMetadata['source_identifier']==sensor['source_id'])
    #print type(sensor['source_id'])
    metadata = labeledMetadata.loc[labeledMetadata['source_identifier']==sensor['source_id']]['point_type'].iloc[0]
    #print(type(metadata))
    typeTestLabels.append(metadata)
    #typeTestLabels.append(metadata['point_type']).iloc[0]
#print typeTestLabels

#srcidList = ebu3bSrcidList
indexList = list()
for srcid in samplePointList:
    index = indexList.append(find_index_sensor_list(sensor_list, srcid))
learningSampleBow = bowStrArray[indexList]
learningSampleBow = scipy.sparse.coo_matrix(learningSampleBow)

typeRawTruthList = list()
for srcid in samplePointList:
    typeRawTruthList.append(labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]['point_type'].iloc[0])
typeLearningLabels = labelEncoder.transform(typeRawTruthList)
for index, raw, label in zip(indexList, typeRawTruthList, typeLearningLabels):
    print index, ": ", raw, ": ", label
#model = AdaBoostClassifier(n_estimators=100)
model = RandomForestClassifier(n_estimators=400, random_state=0)
#model = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
#model = DPGMM(n_components = 17, covariance_type='full')
#model = KNeighborsClassifier(n_neighbors=3)
#model = LogisticRegression()
#model = AdaBoostClassifier(n_estimators=100)
#model = MultinomialNB()
model.fit(X=learningSampleBow.toarray(), y=typeLearningLabels)
predProbs = model.predict_proba(bowStrArray)
predLabels = model.predict(bowStrArray).tolist()

2915 :  actual cooling setpoint :  3
2916 :  actual heating setpoint :  4
2917 :  supply air flow :  147
2918 :  warm cool adjust :  187
2919 :  zone temperature :  190
3408 :  occupied cooling maximum flow :  111
3409 :  damper command :  37
3410 :  occupied heating maximum flow :  114
3411 :  heating valve command :  82
3412 :  occupied cooling setpoint :  113
3413 :  occupied cooling minimum flow :  112
3414 :  occupied heating minimum flow :  115
3415 :  occupied heating setpoint :  116
3416 :  supply air flow setpoint :  148
4029 :  occupied command :  110
1467 :  actual cooling setpoint :  3
1468 :  actual heating setpoint :  4
1469 :  damper position :  39
1470 :  supply air flow :  147
1471 :  warm cool adjust :  187
1472 :  zone temperature :  190
2475 :  cooling maximum flow :  32
2476 :  temperature setpoint :  171
2477 :  heating valve command :  82
2478 :  occupied cooling minimum flow :  112
2479 :  occupied heating minimum flow :  115
2480 :  supply air flow setpoint :  

In [31]:
indices = [1911, 1912, 2242]
for idx in indices:
    print "========"
    print sensor_list[idx]
    print labeledMetadata.loc[labeledMetadata['source_identifier']==sensor_list[idx]['source_id']]
    print predProbs[idx]
    print predLabels[idx]
#filteredPredLabels = [label if prop>=0.4 else None for (label, prop) in zip(predLabels, predProbs)]

{'type_string': 'Analog Output', 'name': 'NAE 13 N2 Trunk 1 VAV 34 HMAX STPT', 'jci_name': 'AP&M.RM-B432C.HMAX-STPT', 'source_id': '513_1_3005325', 'type': 1, 'unit': 84, 'description': 'Occupied Heating Maximum Flow'}
     equip_type  location                     point_type source_identifier
1049        VAV  RM-B432C  occupied heating maximum flow     513_1_3005325
[ 0.      0.      0.0075  0.0025  0.0025  0.02    0.0275  0.04    0.0125
  0.0325  0.64    0.11    0.045   0.0075  0.0125  0.04    0.      0.      0.    ]
114
{'type_string': 'Analog Output', 'name': 'NAE 13 N2 Trunk 1 VAV 34 HTG VLV', 'jci_name': 'AP&M.RM-B432C.HTG-VLV', 'source_id': '513_1_3005326', 'type': 1, 'unit': 98, 'description': 'Heating Valve Command'}
     equip_type  location             point_type source_identifier
1050        VAV  RM-B432C  heating valve command     513_1_3005326
[ 0.      0.      0.      0.04    0.0075  0.8675  0.0125  0.      0.
  0.0075  0.01    0.0075  0.005   0.015   0.0025  0.015   0.  

In [32]:
# Filter prediction result
thresholdProb = 0.5
#print len(predProbs), predProbs.size
for i, (prob, label) in enumerate(zip(predProbs, predLabels)):
    if max(prob) < thresholdProb:
        predLabels[i] = None
    
print(len(predLabels))
print(len(typeTruthLabels))
print np.sum(np.equal(predLabels, None))
decodedPredLabels = [labelEncoder.inverse_transform(label) if label else None for label in predLabels]
correctLabelsNum = (np.sum([(testLabel==predLabel) and (testLabel!=None) for testLabel, predLabel in zip(typeTruthLabels, decodedPredLabels)]))
print "precision for labeled points: ", float(correctLabelsNum)/float(float(len(predLabels)-np.sum(np.equal(predLabels, None))))


4170
4170
902
precision for labeled points:  0.999694002448


In [33]:
# Print and export results


def output_metadata_from_incorrect_dict(incorrectResult):
    resultList = list()
    for srcid, incorrectLabel in incorrectResult.iteritems():
        labeledSensor = labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]
        correctLabel = labeledSensor['location'].get_values()[0]
        #print sensor_df.loc[sensor_df['source_id']==srcid]
        row = sensor_df.loc[sensor_df['source_id']==srcid]#[1]
        oneResult = [srcid,
                     row['jci_name'].values[0],
                     row['name'].values[0],
                     incorrectLabel,
                     correctLabel]     
        resultList.append(oneResult)
    try:
        resultDF = pd.DataFrame(resultList, columns=['source_identifier', 'jci_name', 'name', 'incorrect clustering', 'correct location'])
    except:
        resultDF = pd.DataFrame()
    return resultDF

def output_metadata_from_srcids(srcidList):
    resultList = list()
    for srcid in srcidList:
        labeledSensor = labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]
        correctLabel = labeledSensor['location'].get_values()[0]
        row = sensor_df.loc[sensor_df['source_id']==srcid]#[1]
        oneResult = [srcid,
                     row['jci_name'].values[0],
                     row['name'].values[0],
                     None,
                     correctLabel]     
        resultList.append(oneResult)
    try:
        resultDF = pd.DataFrame(resultList, columns=['source_identifier', 'jci_name', 'name', 'incorrect clustering', 'correct location'])
    except:
        resultDF = pd.DataFrame()
    return resultDF

def output_metadata_from_cluster_dict(clusterDict=None, vavDict=None, vavScoreDict=None, tsDict=None):
    resultList = list()
    for clusterNum, clusterList in clusterDict.iteritems():
        for srcid in clusterList:
            labeledSensor = labeledMetadata.loc[labeledMetadata['source_identifier']==srcid]
            correctLabel = labeledSensor['location'].get_values()[0]
            if vavDict!=None:
                correctEquipLabel = labeledSensor['equip_type'].get_values()[0]
                vavPredict = vavDict[clusterNum]
            else:
                correctEquipLabel = ''
                vavPredict = ''
            if tsDict!=None:
                correctTypeLabel = labeledSensor['point_type'].iloc[0]
                tsPredict = tsDict[srcid]
            else:
                correctTypeLabel = ''
                tsPredict = ''
            idx = find_index_sensor_list(sensor_list, srcid)
            bow = bowNumArray[idx]
            bowStr = str(bow[bow>0])
            bowSum = np.sum(bow)
            if vavScoreDict != None:
                vavScore = str(vavScoreDict[clusterNum])
            else:
                vavScore = ''
            
            
            row = sensor_df.loc[sensor_df['source_id']==srcid]#[1]
            oneResult = [srcid,
                         row['jci_name'].values[0],
                         row['name'].values[0],
                         clusterNum,
                         correctLabel,
                         vavPredict,
                         correctEquipLabel,
                         vavScore,
                         bowStr,
                         bowSum,
                         tsPredict,
                         correctTypeLabel
                        ]     
            resultList.append(oneResult)
    resultDF = pd.DataFrame(resultList, columns=['source_identifier', 'jci_name', 'name', 'cluster number', 'correct location', 'vav predict', 'correct equip type', 'vav pred score', 
                                                 'bow string', 'sum of bow', 'TS preddict', 'true point type'])
    return resultDF


precision = float(totalCnt-len(incorrectResult))/float(totalCnt)
vScore = metrics.v_measure_score(correctLabels, incorrectLabels)
amiScore = metrics.adjusted_mutual_info_score(correctLabels, incorrectLabels)
arScore = metrics.adjusted_rand_score(correctLabels, incorrectLabels)
print vScore, amiScore, arScore
totalNum = len(origSensorList)
nanNum = len(nanList)
zerovectorNum = len(zerovectorList)
filteredNum = len(filteredList)
coverage = float(totalCnt)/float(len(origSensorList))
vavPrecision = float(correctVavCnt)/float(correctVavCnt+incorrectVavCnt)
vavRecall = float(correctVavCnt)/float(correctVavCnt+incorrectNonVavCnt)
vavAccuracyPerPoint = float(vavAccuratePoints) / len(sensor_list)
print buildingName
print "Total Number: ", totalNum
print "V Measure Score: ", vScore
print "Coverage: ", coverage
print "Accuracy: ", precision
print "nan numbers: ", nanNum
print "zero vector numbers: ", zerovectorNum
#print "filtered numbers: ", filteredList
print "VAV precision: ", vavPrecision
print "VAV recall: ", vavRecall
print "VAV accuracy per point: ", vavRecall

summaryDict = {'Total Number' : totalNum,
                 "V Measure score": vScore,
                 "Custom Precision" : precision,
                 "Coverage" : coverage,
                 "non-labeled points" : nanNum,
                 "zero vector points" : zerovectorNum,
                 "filtered (nan-labeled || filtered) points" : filteredNum,
                "VAV precision": vavPrecision,
                "VAV recall": vavRecall,
               "VAV accuracy per point": vavRecall,
               "TS precision": tsPrecision,
               'TS recall: ': tsRecall
                }
summaryDict = OrderedDict(summaryDict)
print summaryDict
summaryDF = pd.DataFrame(data=[summaryDict.values()], columns=summaryDict.keys())
incorrectResultDF = output_metadata_from_incorrect_dict(incorrectResult)
zerovectorResultDF = output_metadata_from_srcids(zerovectorList)
nonlabeledResultDF = output_metadata_from_srcids(nanList)
clusterResultDF = output_metadata_from_cluster_dict(clusterDict=clusterDict, vavDict=vavDict, vavScoreDict=vavScoreDict, tsDict = tsPredictDict)
writer = pd.ExcelWriter(resultFile)
summaryDF.to_excel(writer, 'summary', index=False)
incorrectResultDF.to_excel(writer, 'Incorrect Labels', index=False)
nonlabeledResultDF.to_excel(writer, 'Nonlabeled Labels', index=False)
zerovectorResultDF.to_excel(writer, 'Zero-vector Labels', index=False)
clusterResultDF.to_excel(writer, 'Clusters', index=False)
writer.save()

0.966243355446 0.90743244833 0.783219517667
ap_m
Total Number:  4380
V Measure Score:  0.966243355446
Coverage:  0.952054794521
Accuracy:  0.823501199041
nan numbers:  210
zero vector numbers:  0
VAV precision:  0.93085106383
VAV recall:  0.945945945946
VAV accuracy per point:  0.945945945946
OrderedDict([('TS recall: ', 0.9903846153846154), ('VAV recall', 0.9459459459459459), ('VAV precision', 0.9308510638297872), ('Custom Precision', 0.8235011990407674), ('Total Number', 4380), ('VAV accuracy per point', 0.9459459459459459), ('filtered (nan-labeled || filtered) points', 210), ('V Measure score', 0.96624335544551776), ('zero vector points', 0), ('Coverage', 0.952054794520548), ('non-labeled points', 210), ('TS precision', 1.0)])
